In [1]:
import pandas as pd
import numpy as np
 
from goatools.base import download_go_basic_obo
from goatools.obo_parser import GODag 

import mygene

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

import os
import requests

In [2]:
mfdf = pd.read_csv('../data/ontology/go_annotations_molecular_function.csv')
mfdf.head()


,Gene,Gene_Symbol,Type,GO_ID,Term,Evidence,Category
0,ENSG00000117410,ATP6V0B,Molecular_Function,GO:0005515,protein binding,IPI,MF
1,ENSG00000117410,ATP6V0B,Molecular_Function,GO:0015078,proton transmembrane transporter activity,IEA,MF
2,ENSG00000117410,ATP6V0B,Molecular_Function,GO:0046961,"proton-transporting ATPase activity, rotationa...",IEA,MF
3,ENSG00000117410,ATP6V0B,Molecular_Function,GO:0046961,"proton-transporting ATPase activity, rotationa...",TAS,MF
4,ENSG00000204520,MICA,Molecular_Function,GO:0005515,protein binding,IPI,MF


In [3]:
unique_genes = mfdf['Gene'].unique().tolist()

In [4]:
len(unique_genes)

3437

In [41]:
def get_versioned_ontology():
    # 1. Define the local target name (Versioned)
    today = datetime.now().strftime('%Y-%m-%d')
    local_filename = f"go-basic_{today}.obo"
    
    # 2. Define the static source URL (Always the same)
    url = "http://purl.obolibrary.org/obo/go/go-basic.obo"
    
    # 3. Check if we already have it locally
    if os.path.exists(local_filename):
        print(f"Loading existing ontology: {local_filename}")
    else:
        print(f"Downloading new ontology from {url}...")
        print(f"Saving locally as {local_filename}...")
        
        # Manual Download to control the filename
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(local_filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        f.write(chunk)
        else:
            raise ValueError(f"Server returned {response.status_code}. Could not download OBO file.")
        
    # 4. Load the DAG
    # load_obsolete=True is CRITICAL to find old IDs
    return GODag(local_filename, load_obsolete=True)

In [ ]:
# --- Initialize the DAG ---
#go_dag = get_versioned_ontology()

In [5]:
# get go_dag from the saved filename
go_dag = GODag('go-basic_2025-11-28.obo', load_obsolete=True, optional_attrs=['relationship'])

go-basic_2025-11-28.obo: fmt(1.2) rel(2025-10-10) 51,842 Terms; optional_attrs(relationship)


In [3]:
count_with_relations = 0
example_id = None
example_type = None

print("Scanning graph for relationships...")
for term in go_dag.values():
    if hasattr(term, 'relationship') and term.relationship:
        count_with_relations += 1
        if example_id is None:
            example_id = term.id
            example_type = list(term.relationship.keys())[0]

if count_with_relations == 0:
    print("\n[ERROR] The graph has ZERO relationships.")
    print("Check your get_versioned_ontology() function.")
    print("Make sure it has: optional_attrs=['relationship']")
else:
    print(f"\n[SUCCESS] Found {count_with_relations} terms with relationships.")
    print(f"Example: {example_id} has a '{example_type}' relation.")
    
    # 3. Print the example to prove it works
    term = go_dag[example_id]
    print(f"\n--- Details for {term.name} ---")
    print(f"Parents (is_a): {[p.id for p in term.parents]}")
    print(f"Relationships: {term.relationship}")

Scanning graph for relationships...

[SUCCESS] Found 15046 terms with relationships.
Example: GO:0000015 has a 'part_of' relation.

--- Details for phosphopyruvate hydratase complex ---
Parents (is_a): ['GO:1902494']
Relationships: {'part_of': {GOTerm('GO:0005829'):
  id:GO:0005829
  item_id:GO:0005829
  name:cytosol
  namespace:cellular_component
  _parents: 1 items
    GO:0110165
  parents: 1 items
    GO:0110165	level-01	depth-01	cellular anatomical structure [cellular_component]
  children: 1 items
    GO:0099522	level-03	depth-03	cytosolic region [cellular_component]
  level:2
  depth:2
  is_obsolete:False
  alt_ids: 0 items
  relationship: 1 items
    part_of: 1 items
      GO:0005737	level-02	depth-02	cytoplasm [cellular_component]
  relationship_rev: 1 items
    part_of: 44 items
      GO:0072379	level-02	depth-02	ER membrane insertion complex [cellular_component]
      GO:1990588	level-03	depth-03	FtsBL complex [cellular_component]
      GO:0005832	level-04	depth-04	chaperonin

In [ ]:
# 2. Counters
total_terms = 0
terms_with_relations = 0
relation_types_found = set()

print("Auditing Graph...")

for term in go_dag.values():
    total_terms += 1
    if hasattr(term, 'relationship') and term.relationship:
        terms_with_relations += 1
        # Record what types we found (regulates, part_of, etc.)
        for key in term.relationship.keys():
            relation_types_found.add(key)

print(f"\nTotal Terms in DAG: {total_terms}")
print(f"Terms with Typed Relationships: {terms_with_relations}")
print(f"Relationship Types Found: {relation_types_found}")

if terms_with_relations == 0:
    print("\n[DIAGNOSIS]: Your DAG object DOES NOT have relationships loaded.")
    print("Please check your get_versioned_ontology function ensures optional_attrs=['relationship']")
else:
    print("\n[DIAGNOSIS]: The DAG is fine! Your genes simply map to terms that only use 'is_a' logic.")

Auditing Graph...

Total Terms in DAG: 51842
Terms with Typed Relationships: 15046
Relationship Types Found: {'regulates', 'part_of', 'negatively_regulates', 'positively_regulates'}

[DIAGNOSIS]: The DAG is fine! Your genes simply map to terms that only use 'is_a' logic.


In [4]:
# Ensure your DAG is loaded with relationships!
# go_dag = get_versioned_ontology() 

def get_regulatory_targets(go_id):
    if go_id not in go_dag:
        print(f"ID {go_id} not found.")
        return

    term = go_dag[go_id]
    print(f"Checking: {term.name} ({term.id})")
    
    # These are the 3 keys that represent regulation
    reg_keys = ['regulates', 'positively_regulates', 'negatively_regulates']
    found_any = False

    if hasattr(term, 'relationship'):
        for key in reg_keys:
            if key in term.relationship:
                targets = term.relationship[key]
                for target in targets:
                    print(f"  [{key}] -> {target.name} ({target.id})")
                    found_any = True
    
    if not found_any:
        print("  This term does not regulate anything (or relationships weren't loaded).")

# --- TEST CASE ---
# "Regulation of apoptotic process" (GO:0042981) should regulate "Apoptotic process"
get_regulatory_targets("GO:0042981")

Checking: regulation of apoptotic process (GO:0042981)
  [regulates] -> apoptotic process (GO:0006915)


In [71]:
go_dag['GO:0006915']

GOTerm('GO:0006915'):
  id:GO:0006915
  item_id:GO:0006915
  name:apoptotic process
  namespace:biological_process
  _parents: 1 items
    GO:0012501
  parents: 1 items
    GO:0012501	level-03	depth-03	programmed cell death [biological_process]
  children: 19 items
  level:4
  depth:4
  is_obsolete:False
  alt_ids: 2 items
    GO:0008632
    GO:0006917
  relationship: 0 items
  relationship_rev: 4 items
    part_of: 3 items
      GO:0097194	level-02	depth-02	execution phase of apoptosis [biological_process]
      GO:0097190	level-03	depth-05	apoptotic signaling pathway [biological_process]
      GO:0008637	level-06	depth-06	apoptotic mitochondrial changes [biological_process]
    regulates: 1 items
      GO:0042981	level-05	depth-05	regulation of apoptotic process [biological_process]
    positively_regulates: 1 items
      GO:0043065	level-06	depth-06	positive regulation of apoptotic process [biological_process]
    negatively_regulates: 1 items
      GO:0043066	level-06	depth-06	nega

In [6]:
def get_term_metrics(go_id):
    # Case 1: ID exists in the OBO file
    if go_id in go_dag:
        term = go_dag[go_id]
        return term.level, term.depth
    
    # Case 2: ID is missing (Deleted/Merged and not in this OBO version)
    # We return 0, 0 so your code continues without crashing.
    # You can filter these out later by checking if Level == 0.
    return 0, 0

In [7]:
not_found_genes = []
genes_with_no_mf, genes_with_no_bp = [], []

def fetch_go_data(gene_list):
    mg = mygene.MyGeneInfo()
    
    # 1. Query fields
    # We ask for go.BP (Process) and go.MF (Function)
    results = mg.querymany(gene_list, 
                           scopes='ensembl.gene', 
                           fields='symbol,go.BP,go.MF', 
                           species='human')
    
    bp_data, mf_data = [], []
    
    for item in results:
        gene = item.get('query')
        symbol = item.get('symbol', 'Unknown')
        # Skip if no GO data found
        if 'go' not in item:
            not_found_genes.append(gene)
            continue
            
        # 2. Extract Biological Process (BP) - "The Big Picture"
        # We normalize because sometimes it returns a dict (1 result) or list (many results)
        bp_raw = item['go'].get('BP', [])
        if isinstance(bp_raw, list) and len(bp_raw) == 0:
            genes_with_no_bp.append(gene)
        if isinstance(bp_raw, dict): bp_raw = [bp_raw]
        
        for entry in bp_raw:
            go_id = entry.get('id')
            level, depth = get_term_metrics(go_id)
            bp_data.append({
                'Gene': gene,
                'Gene_Symbol': symbol,
                'Type': 'Biological_Process',
                'GO_ID': go_id,
                'Term': entry.get('term'),
                'Evidence': entry.get('evidence'),
                'Category': 'BP',
                'Level': level,
                'Depth': depth
            })
            
        # 3. Extract Molecular Function (MF) - "The Mechanism"
        mf_raw = item['go'].get('MF', [])
        if isinstance(mf_raw, list) and len(mf_raw) == 0:
            genes_with_no_mf.append(gene)
        if isinstance(mf_raw, dict): mf_raw = [mf_raw]
        
        for entry in mf_raw:
            go_id = entry.get('id')
            level, depth = get_term_metrics(go_id)
            mf_data.append({
                'Gene': gene,
                'Gene_Symbol': symbol,
                'Type': 'Molecular_Function',
                'GO_ID': go_id,
                'Term': entry.get('term'),
                'Evidence': entry.get('evidence'),
                'Level': level,
                'Depth': depth,
                'Category': 'MF'
            })

    return pd.DataFrame(bp_data), pd.DataFrame(mf_data)

In [9]:
bp_df, mf_df = fetch_go_data(unique_genes)
bp_df.head()

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed


,Gene,Gene_Symbol,Type,GO_ID,Term,Evidence,Category,Level,Depth
0,ENSG00000117410,ATP6V0B,Biological_Process,GO:0006811,monoatomic ion transport,IEA,BP,4,4
1,ENSG00000117410,ATP6V0B,Biological_Process,GO:0007035,vacuolar acidification,NAS,BP,5,9
2,ENSG00000117410,ATP6V0B,Biological_Process,GO:0007042,lysosomal lumen acidification,NAS,BP,5,10
3,ENSG00000117410,ATP6V0B,Biological_Process,GO:0016241,regulation of macroautophagy,NAS,BP,7,7
4,ENSG00000117410,ATP6V0B,Biological_Process,GO:0048388,endosomal lumen acidification,NAS,BP,5,9


In [10]:
len(genes_with_no_bp), len(genes_with_no_mf)

(120, 0)

In [11]:
bp_df.Gene.nunique()

3317

In [ ]:
#bp_df.to_csv('../data/ontology/go_annotations_biological_process_{}.csv'.format(pd.Timestamp.now().strftime('%Y-%m-%d')), index=False)

In [13]:
p = 3437 - 120 # number of unique genes to be annotated - genes with no BP in the GO 
print(f"Percentage of the genes without BP: {120 / 3437:.2%}") 


Percentage of the genes without BP: 3.49%


In [14]:
mf_df.shape

(27091, 9)

In [15]:
bp_df.shape

(41949, 9)

In [16]:
bp_df.Level.value_counts()

Level
5     9280
6     7820
4     6882
7     6070
3     3715
8     3225
9     1881
2     1281
10    1163
11     362
1      158
0       86
13      15
12      11
Name: count, dtype: int64

In [62]:
mf_df.Level.value_counts()

Level
3    6693
2    6514
4    5482
5    4776
6    1921
7    1267
1     332
8      89
9      10
0       7
Name: count, dtype: int64

In [63]:
mf_df.Depth.value_counts()

Depth
2     6116
3     5072
4     4664
5     4320
6     2587
7     2364
8     1416
1      332
9      139
11      40
10      34
0        7
Name: count, dtype: int64

In [46]:
mf_df.shape

(27091, 9)

In [17]:
#mf_df.to_csv('../data/ontology/go_annotations_molecular_{}.csv'.format(pd.Timestamp.now().strftime('%Y-%m-%d')), index=False)
mf_df = pd.read_csv("../data/ontology/go_annotations_molecular_2025-11-28.csv", index_col=None)
mf_df.head()

,Gene,Gene_Symbol,Type,GO_ID,Term,Evidence,Level,Depth,Category
0,ENSG00000117410,ATP6V0B,Molecular_Function,GO:0005515,protein binding,IPI,2,2,MF
1,ENSG00000117410,ATP6V0B,Molecular_Function,GO:0015078,proton transmembrane transporter activity,IEA,5,5,MF
2,ENSG00000117410,ATP6V0B,Molecular_Function,GO:0046961,"proton-transporting ATPase activity, rotationa...",IEA,4,8,MF
3,ENSG00000117410,ATP6V0B,Molecular_Function,GO:0046961,"proton-transporting ATPase activity, rotationa...",TAS,4,8,MF
4,ENSG00000204520,MICA,Molecular_Function,GO:0005515,protein binding,IPI,2,2,MF


In [21]:
bp_df = pd.read_csv("../data/ontology/go_annotations_biological_process_2025-11-30.csv", index_col=None)
bp_df.head()

,Gene,Gene_Symbol,Type,GO_ID,Term,Evidence,Category,Level,Depth
0,ENSG00000117410,ATP6V0B,Biological_Process,GO:0006811,monoatomic ion transport,IEA,BP,4,4
1,ENSG00000117410,ATP6V0B,Biological_Process,GO:0007035,vacuolar acidification,NAS,BP,5,9
2,ENSG00000117410,ATP6V0B,Biological_Process,GO:0007042,lysosomal lumen acidification,NAS,BP,5,10
3,ENSG00000117410,ATP6V0B,Biological_Process,GO:0016241,regulation of macroautophagy,NAS,BP,7,7
4,ENSG00000117410,ATP6V0B,Biological_Process,GO:0048388,endosomal lumen acidification,NAS,BP,5,9


In [22]:
bp_df.Gene.nunique()

3317

In [18]:
mf_df.Gene.nunique()

3437

In [33]:
def get_regulatory_targets(go_id):
    """
    Finds all regulatory targets for a given GO ID.
    
    This modified version returns all targets and uses a consistent
    return type to make it more robust and easier to use.
    """
    if go_id not in go_dag:
        print(f"ID {go_id} not found.")
        # Always return a tuple with a list to be consistent
        return None, []

    term = go_dag[go_id]
    found_targets = []
    
    # These are the 3 keys that represent regulation
    reg_keys = ['regulates', 'positively_regulates', 'negatively_regulates']

    if hasattr(term, 'relationship'):
        for key in reg_keys:
            if key in term.relationship:
                # A relationship can have one or multiple targets. 
                # We normalize to a list to handle both cases.
                targets = term.relationship[key]
                if not isinstance(targets, (list, set)):
                    targets = [targets]

                for target in targets:
                    #print(f"  [{key}] -> {target.name} ({target.id})")
                    # Store all relevant info for each target found
                    found_targets.append({
                        'regulator_id': term.id,
                        'regulator_name': term.name,
                        'regulation_type': key,
                        'target_id': target.id,
                        'target_name': target.name
                    })
    
    # Return the original term and the list of all targets found
    return term, found_targets


In [35]:
unique_terms = mf_df['Term'].unique().tolist()
unique_term_IDs = mf_df['GO_ID'].unique().tolist()
print("Total unique terms:", len(unique_terms))
print("Total unique term IDs:", len(unique_term_IDs))

Total unique terms: 2147
Total unique term IDs: 2147


In [38]:
# Initialize lists to store the results for MF
mf_regulatory_relationships = []
mf_go_id_not_found = []

# Loop over the unique Molecular Function GO IDs
for go_id in unique_term_IDs:
    term, found_targets = get_regulatory_targets(go_id)
    
    # If the term is None, the ID was not found in the DAG
    if term is None:
        mf_go_id_not_found.append(go_id)
    
    # If the list of found_targets is not empty, extend our main list with its contents
    elif found_targets:
        mf_regulatory_relationships.extend(found_targets)

print(f"For MF: Found {len(mf_regulatory_relationships)} total regulatory relationships.")
print(f"For MF: Could not find {len(mf_go_id_not_found)} GO IDs in the DAG.")

For MF: Found 0 total regulatory relationships.
For MF: Could not find 0 GO IDs in the DAG.


In [41]:
unique_bp_terms = bp_df['Term'].unique().tolist()
unique_bp_term_IDs = bp_df['GO_ID'].unique().tolist()
print("Total unique biological process terms:", len(unique_bp_terms))
print("Total unique biological process term IDs:", len(unique_bp_term_IDs))

Total unique biological process terms: 6435
Total unique biological process term IDs: 6435


In [43]:
# Initialize lists to store the results for BP
bp_regulatory_relationships = []
bp_go_id_not_found = []

# Loop over the unique Biological Process GO IDs
for go_id in unique_bp_term_IDs:
    term, found_targets = get_regulatory_targets(go_id)
    
    # If the term is None, the ID was not found in the DAG
    if term is None:
        bp_go_id_not_found.append(go_id)
    
    # If the list of found_targets is not empty, extend our main list with its contents
    elif found_targets:
        bp_regulatory_relationships.extend(found_targets)

print(f"For BP: Found {len(bp_regulatory_relationships)} total regulatory relationships.")
print(f"For BP: Could not find {len(bp_go_id_not_found)} GO IDs in the DAG.")

For BP: Found 2068 total regulatory relationships.
For BP: Could not find 0 GO IDs in the DAG.


In [37]:
bp_regulatory_relationships[:5]

[{'regulator_id': 'GO:0016241',
  'regulator_name': 'regulation of macroautophagy',
  'regulation_type': 'regulates',
  'target_id': 'GO:0016236',
  'target_name': 'macroautophagy'},
 {'regulator_id': 'GO:0031348',
  'regulator_name': 'negative regulation of defense response',
  'regulation_type': 'negatively_regulates',
  'target_id': 'GO:0006952',
  'target_name': 'defense response'},
 {'regulator_id': 'GO:0032815',
  'regulator_name': 'negative regulation of natural killer cell activation',
  'regulation_type': 'negatively_regulates',
  'target_id': 'GO:0030101',
  'target_name': 'natural killer cell activation'},
 {'regulator_id': 'GO:0045953',
  'regulator_name': 'negative regulation of natural killer cell mediated cytotoxicity',
  'regulation_type': 'negatively_regulates',
  'target_id': 'GO:0042267',
  'target_name': 'natural killer cell mediated cytotoxicity'},
 {'regulator_id': 'GO:0050776',
  'regulator_name': 'regulation of immune response',
  'regulation_type': 'regulates',

In [45]:
# Convert the list of BP relationships into a DataFrame
if bp_regulatory_relationships:
    bp_reg_df = pd.DataFrame(bp_regulatory_relationships)
    print("Biological Process Regulatory Relationships DataFrame:")
else:
    print("No regulatory relationships were found for Biological Process; an empty DataFrame will be created.")
    bp_reg_df = pd.DataFrame()

Biological Process Regulatory Relationships DataFrame:


In [46]:
bp_reg_df.head()

,regulator_id,regulator_name,regulation_type,target_id,target_name
0,GO:0016241,regulation of macroautophagy,regulates,GO:0016236,macroautophagy
1,GO:0031348,negative regulation of defense response,negatively_regulates,GO:0006952,defense response
2,GO:0032815,negative regulation of natural killer cell act...,negatively_regulates,GO:0030101,natural killer cell activation
3,GO:0045953,negative regulation of natural killer cell med...,negatively_regulates,GO:0042267,natural killer cell mediated cytotoxicity
4,GO:0050776,regulation of immune response,regulates,GO:0006955,immune response


In [47]:
bp_reg_df.shape

(2068, 5)

In [48]:
bp_reg_df.regulator_id.nunique(), bp_reg_df.target_id.nunique()

(2068, 1180)

In [ ]:
# Save the Biological Process regulatory relationships
'''bp_reg_df.to_csv(
    '../data/ontology/go_regulatory_relationships_biological_process_{}.csv'.format(pd.Timestamp.now().strftime('%Y-%m-%d')), 
    index=False
)'''

In [50]:
regulator_map = bp_reg_df.groupby('regulator_id')['target_id'].apply(list).to_dict()
print(F"Successfuly created a Regulator map with {len(regulator_map)} regulator to target relationship!")

Successfuly created a Regulator map with 2068 regulator to target relationship!
